# **Deep Learning Project**

**Eliana Battisti (223701) - Davide Dalla Stella () - Francesco Trono (221723)**

*University of Trento*

A.Y. 2020/2021 - Deep Learning Course

Click to open & run in Colab:


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ftrono/DL_Project/blob/main/source_code/APRNet_main.ipynb)

In [ ]:
!pip install --pre torch torchvision -f https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html -U

Looking in links: https://download.pytorch.org/whl/nightly/cu102/torch_nightly.html


In [ ]:
import os
import shutil
import csv
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
import collections
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from google.colab import drive
from sklearn.preprocessing import MultiLabelBinarizer

**0) Set working directory & global variables**

In [ ]:
#Set FLAG to 1 only if you want to create the git clone and the Datasets from scratch
FLAG = 1

if flag == 1:
    !git clone https://github.com/ftrono/DL_Project.git
    % cd DL_Project
    print("Git cloned.")
    #unzip dataset:
    !unzip 'dataset.zip' -d './Dataset' >/dev/null
    print("Dataset unzipped.")

In [ ]:
#Global variables:
num_classifiers = 12
device = "cuda:0"
#device = "cpu"
datapath = "./Dataset/"

**1) Dataset preparation**

1.1) Dataset split (training / validation)

In [ ]:
#Dataset split
def dataset_preparation(img_root, csv_dir):

    # PART 1
    validation_dir = img_root+"validation"
    train_dir = img_root+"train"
    n_id = -1 #id counter (starts from -1 to exclude heading)

    #validation folder must be created & populated only once:
    if not os.path.isdir(validation_dir):
        os.mkdir(validation_dir)
        #open annotation csv
        with open(csv_dir) as annotations_train:
            #print(annotations_train)
            #count and sum ids with a for cycle
            n_id += sum(1 for row in annotations_train)

        #split: training 75%, validation 25%
        to_move = n_id // 4

        #move images to validation folder:
        print("Moving 25% of training files to validation")
        for i in range(0, to_move):
            #get updated list of filenames in the train directory (do it at each new loop)
            train_list = os.listdir(train_dir)
            #get person ID from first filename (first 4 chars)
            id_to_move = train_list[0][0:4]
            #print("id to move: ", id_to_move)
            #move all files with that ID:
            for files in train_list:
                if files.startswith(id_to_move):
                    shutil.move(train_dir+"/"+files, validation_dir+"/"+files)
                    #print("Moved: ",train_dir+"/"+files, "to: ",validation_dir+"/"+files)

        print("Validation files moved to val folder.")
    else:
        print("Validation already created.")


    # PART 2
    '''
    validation_queries_dir = img_root+ "/validation_queries"
    test_dir = img_root + "/test"

    if not os.path.isdir(validation_queries_dir): #Se una delle cartelle non esiste allora assumo che non ne esista nessuna
      if not os.path.isdir(test_dir):
        os.mkdir(validation_queries_dir) #creazione cartelle validation_query e test
        os.mkdir(test_dir)
        validation_list = os.listdir(validation_dir) #lista di file all'interno della cartella validation
        copied_id = [] #lista degli id che ho già copiato in validation list. All'inizio è vuota perchè non ho copiato nessuna immagine
        for file in validation_list:  #scorro la lista di file di validation
          id = file[0:4] #prelevo l'id del file corrente
          if id in copied_id: #se l'id è nell'array allora l'ho già copiato in validation_query quindi quest'altra immagine va copiata in test
            shutil.copy(validation_dir+"/"+file,test_dir+"/"+file) #copia file
          else:
          #se l'id non è presente nell'array allora devo copiare l'immagine in validation_query e inserire l'id nell'array dei già copiati
            shutil.copy(validation_dir+"/"+file,validation_queries_dir+"/"+file)
            copied_id.append(id)
    '''

1.2) Dataset class (stores the samples and their corresponding labels):

In [ ]:
class CustomDataset(Dataset):

    #override:
    #init dataset class:
    #(Note: csvfile and imgfolder are directory strings)
    def __init__(self, imgfolder, train, csvfile=None):

        #init annotations dictionary:
        self.train = train
        self.imgfolder = imgfolder
        self.dictionary = {}
        #number of files in the directory
        self.img_list = os.listdir(imgfolder)
        self.size = len(self.img_list)
        #if annotations available:
        if csvfile != None:
            with open(csvfile, mode='r') as annotations_train:
                reader = csv.reader(annotations_train, delimiter=',')
                #skip header row
                next(reader, None)
                for row in reader:
                    #group upcolor values (8 labels):
                    up = row[11:19]
                    #group downcolor values (9 labels):
                    down = row[19:]
                    #get id:
                    id = int(row[0])

                    #init attributes concatenation with first 10 attributes:
                    row = row[1:11]

                    #append upcolor value to concatenation (set as label index+1):
                    #NOTE: if no original value in upcolors is '2', the label must be 'multicolor' (index = 9):
                    if '2' in up:
                        row.append(up.index('2')+1)
                    else:
                        row.append(9)

                    #append downcolor value to concatenation (same as for upcolor. Multicolor index here will be 10):   
                    if '2' in down:
                        row.append(down.index('2')+1)
                    else:
                        row.append(10)

                    #convert all labels to int - 1:
                    for label in row:
                        row[row.index(label)] = int(label) - 1

                    #append to annotations dictionary:
                    self.dictionary[id] = row

    #override:
    #return the element at index idx:
    def __getitem__(self, idx):
        #standard transformations for the data loader:
        transform = list()
        #normalize with ImageNet mean:
        transform.append(T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225])
        )
        #resize:
        transform.append(T.Resize((256, 128)))

        #get image and convert to tensor, then apply standard transformations:
        img = Image.open(self.imgfolder+"/"+self.img_list[idx])
        img = T.ToTensor()(img)
        transform = T.Compose(transform)
        img = transform(img)

        #if training mode:    
        if self.train == True:
            #apply additional transformations for the train loader:
            transform = list()
            transform.append(T.RandomRotation(5))
            transform.append(T.RandomCrop((256, 128), 10))
            transform.append(T.RandomHorizontalFlip())
            transform = T.Compose(transform)
            img = transform(img)

            #return image & labels:
            #print(self.dictionary[int(self.img_list[idx][0:4])])
            return img, (torch.as_tensor(self.dictionary[int(self.img_list[idx][0:4])]))
        else:
            #return only image:
            return img
    
    #override:
    #return the number of elements that compose the dataset:
    def __len__(self):
        return self.size

1.3) Dataloader class (wraps an iterable around the Dataset to enable easy access to the samples):

In [ ]:
def get_data(batch_size, img_root):

    #load data:
    training_data = CustomDataset(img_root+"train", True, img_root+"annotations_train.csv")
    val_data = CustomDataset(img_root+"validation", True, img_root+"annotations_train.csv")
    test_data = CustomDataset(img_root+"test", False)
    query_data = CustomDataset(img_root+"queries", False)

    #initialize dataloaders:
    train_loader = torch.utils.data.DataLoader(training_data, batch_size, shuffle=True, num_workers=4)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size, shuffle=False, num_workers=4)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size, shuffle=False, num_workers=4)
    query_loader = torch.utils.data.DataLoader(query_data, batch_size, shuffle=False, num_workers=4)

    return train_loader, val_loader, test_loader, query_loader

1.4) Attributes translator (from integers to human-readable strings):

In [ ]:
def translate_attributes(attributes):
  translated_attributes = {}
  attributes_names = ["age","backpack","bag","handbag","clothes","down","up","hair","hat","gender","upcolor","downcolor"] #translator

  translator = [
                ["young", "teenager", "adult", "old"], #index: 0 attribute: age
                ["no", "yes"], #index: 1 attribute: backpack
                ["no", "yes"], #index: 2 attribute: bag
                ["no", "yes"], #index: 3 attribute: handbag
                ["dress", "pants"], #index: 4 attribute: clothes
                ["long lower body clothing", "short"], #index: 5 attribute: down
                ["long sleeve short", "sleeve"], #index: 6 attribute: up
                ["short hair", "long hair"], #index: 7 attribute: hair
                ["no", "yes"], #index: 8 attribute: hat
                ["male","female"], #index: 9 attribute: gender
                ["black", "white", "red", "purple", "yellow", "gray", "blue", "green", "multicolor"], #index: 10 attribute: upcolor
                ["black", "white", "pink", "purple", "yellow", "gray", "blue", "green", "brown", "multicolor"], #index: 11 attribute: downcolor
  ]
  
  #attributes positions are the same in the 2 vectors: 
  #to translate a label, simply do value-1 to get its index in the translated labels list:
  for i in range(len(attributes)):
    translated_attributes[attributes_names[i]] = translator[i][attributes[i]-1] 
  return translated_attributes

**2) CNN implementation**

2.1) Net:

In [ ]:
class Our_CNN(torch.nn.Module):

    #init override:
    def __init__(self, num_heads=num_classifiers, loss={'xent'}, **kwargs):
        super(Our_CNN, self).__init__()

        #load default ResNet from PyTorch:
        resnet = torchvision.models.resnet50(pretrained=True, progress=True)
        #save number of input features from last layer:
        last_infeats = resnet.fc.in_features

        #remove last FC layer:
        self.backbone = torch.nn.Sequential(*list(resnet.children())[:-1])

        #append list of classifiers (one for each attribute):
        self.fc = torch.nn.ModuleList()

        self.fc.append(torch.nn.Linear(last_infeats, 4)) #age classifier
        for i in range(1, 10):
            self.fc.append(torch.nn.Linear(last_infeats, 2)) #binary classifiers
        self.fc.append(torch.nn.Linear(last_infeats, 9)) #upcolor classifier
        self.fc.append(torch.nn.Linear(last_infeats, 10)) #downcolor classifier


    # forward pass:
    def forward(self, x):
        #forward though backbone portion of network:
        x = self.backbone(x)
        x = x.flatten(1)

        #put the output in (batch_size, input_dim) format and save as features:
        feats = x.view(x.shape[0], -1)

        #loop through classifiers and store fwd pass in outputs list:
        outputs = []
        for fc in self.fc:
            outputs.append(fc(x))

        #return both output list (task 1) and features (task 2):
        return outputs, feats

2.2) Loss function (total loss = sum of individual cross-entropy loss for each attribute classifier):

In [ ]:
def cost_function(outputs, targets):
    loss = 0.0
    for i in range(len(outputs)):
        loss += F.cross_entropy(outputs[i], targets.t()[i])
    return loss

2.3) Optimizer:

In [ ]:
def get_optimizer(model, lr, wd, momentum):
  return torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=wd)

2.4) Train & test functions:

In [ ]:
#a) Train function:
def train(net,data_loader,optimizer):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  #training mode:
  net.train()
  for batch_idx, (inputs, targets) in enumerate(data_loader):
    #load data into GPU:
    inputs = inputs.to(device)
    targets = targets.to(device)
    #forward pass:
    outputs, feats = net(inputs)

    #get loss:
    loss = 0
    loss = cost_function(outputs,targets)

    #stats print:
    samples+=inputs.shape[0]
    cumulative_loss += loss.item() #note: the .item() is needed to extract scalars from tensors
    predicted = []
    for i in range(len(outputs)):
      predicted.append(outputs[i].max(1)[1])
      cumulative_accuracy += predicted[i].eq(targets.t()[i]).sum().item()
      
    #backpropagate:
    loss.backward()
    
    #update parameters:
    optimizer.step()
    
    #reset the optimizer:
    optimizer.zero_grad()

  return cumulative_loss/samples, 100*cumulative_accuracy/(len(outputs)*samples)


#b) Test function:
def test(net, data_loader):
  samples = 0.
  cumulative_loss = 0.
  cumulative_accuracy = 0.

  #evaluation mode:
  net.eval()
  with torch.no_grad():
    for batch_idx, (inputs, targets) in enumerate(data_loader):
      #print("batch_idx: ",batch_idx)
      
      #load data into GPU:
      inputs = inputs.to(device)
      targets = targets.to(device)

      #forward pass:
      outputs,feats = net(inputs)
      
      #get loss:
      loss = 0
      loss = cost_function(outputs,targets)

      #stats print:
      samples+=inputs.shape[0]
      cumulative_loss += loss.item() 
      predicted = []
      for i in range(len(outputs)):
        predicted.append(outputs[i].max(1)[1])
        cumulative_accuracy += predicted[i].eq(targets.t()[i]).sum().item()

  return cumulative_loss/samples, 100*cumulative_accuracy/(len(outputs)*samples)

In [ ]:
#Funzione di esportazione file output per task 1 ("classification_test.csv" - vedi PDF assignment)

In [ ]:
#Funzione di esportazione file output per task 2 ("reid_text.txt" - vedi PDF assignment)

**3) Main**

3.1) Main function:

In [ ]:
#Task 1:
'''
Input arguments
  batch_size: Size of a mini-batch
  device: GPU where you want to train your network
  weight_decay: Weight decay co-efficient for regularization of weights
  momentum: Momentum for SGD optimizer
  epochs: Number of epochs for training the network
  num_classes: Number of classes in your dataset
  visualization_name: Name of the visualization folder
  img_root: The root folder of images
'''

def main(batch_size=64, 
         learning_rate=0.001, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=50, 
         num_classes=12, 
         visualization_name='resnet50', 
         img_root=datapath):
  
  #prepare the validation, query_validation and test folders:
  dataset_preparation(img_root,img_root+"annotations_train.csv")

  #logger:
  writer = SummaryWriter(log_dir="./Output/exp1")

  #instantiate dataloaders:
  train_loader, val_loader, test_loader, query_loader = get_data(batch_size=batch_size, img_root=img_root)
  
  #instantiate the network:
  print(torch.cuda.get_device_name(0))
  net = Our_CNN()
  net = net.to(device)
  
  #instantiate the optimizer:
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)

  print('Before training:')
  train_loss, train_accuracy = test(net, train_loader)
  val_loss, val_accuracy = test(net,val_loader)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Val loss {:.5f}, Val accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('-----------------------------------------------------')
  
  #add values to plots:
  writer.add_scalar('Loss/train_loss', train_loss, 0)
  writer.add_scalar('Loss/val_loss', val_loss, 0)
  writer.add_scalar('Accuracy/train_accuracy', train_accuracy, 0)
  writer.add_scalar('Accuracy/val_accuracy', val_accuracy, 0)

  #epochs:
  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer) #train
    val_loss, val_accuracy = test(net, val_loader) #test
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    print('\t Val loss {:.5f}, Val accuracy {:.2f}'.format(val_loss, val_accuracy))
    print('-----------------------------------------------------')
    
    #add values to plots:
    writer.add_scalar('Loss/train_loss', train_loss, e + 1)
    writer.add_scalar('Loss/val_loss', val_loss, e + 1)
    writer.add_scalar('Accuracy/train_accuracy', train_accuracy, e + 1)
    writer.add_scalar('Accuracy/val_accuracy', val_accuracy, e + 1)

  #test:
  print('After training:')
  train_loss, train_accuracy = test(net, train_loader)
  val_loss, val_accuracy = test(net, val_loader)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
  print('\t Val loss {:.5f}, Val accuracy {:.2f}'.format(val_loss, val_accuracy))
  print('-----------------------------------------------------')

  #closes the logger:
  writer.close()

3.2) Reset runs directory & load Tensorboard:

In [ ]:
runs = ".Output/runs"
! rm -r runs

%load_ext tensorboard
%tensorboard --logdir=runs

3.3) Execute:

In [ ]:
main()

<_io.TextIOWrapper name='/content/Dataset/annotations_train.csv' mode='r' encoding='UTF-8'>
Moving 25% of training files in validation
Training files moved to validation
Tesla K80
Before training:
	 Training loss 0.19159, Training accuracy 46.87
	 Val loss 0.19205, Val accuracy 46.91
-----------------------------------------------------
Epoch: 1
	 Training loss 0.09549, Training accuracy 80.33
	 Val loss 0.11555, Val accuracy 78.04
-----------------------------------------------------
Epoch: 2
	 Training loss 0.07056, Training accuracy 85.52
	 Val loss 0.10659, Val accuracy 80.69
-----------------------------------------------------
Epoch: 3
	 Training loss 0.05844, Training accuracy 88.16
	 Val loss 0.11763, Val accuracy 80.89
-----------------------------------------------------
Epoch: 4
	 Training loss 0.04840, Training accuracy 90.14
	 Val loss 0.11966, Val accuracy 80.86
-----------------------------------------------------
Epoch: 5
	 Training loss 0.03996, Training accuracy 92.01

In [ ]:
#MAIN PER TASK 2
#Ripetere da sopra e poi???